In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [45]:
df = pd.read_csv('lyrics_data_cleaned_grouped.csv', sep=';')
df.head()

# keep only lyrics and label columns
df = df[['lyrics', 'sentiment']]



df.dropna(inplace=True)

df.info()

label_mapping = {label: idx for idx, label in enumerate(df['sentiment'].unique())}

df['sentiment'] = df['sentiment'].map(label_mapping)

# replace \n with space
df['lyrics'] = df['lyrics'].str.replace('\n', ' ')

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1557 entries, 0 to 3153
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   lyrics     1557 non-null   object
 1   sentiment  1557 non-null   object
dtypes: object(2)
memory usage: 36.5+ KB


,lyrics,sentiment
1593,I know You've fallen again The way I fell bef...,3
2493,You're not alone I've come to take you home Yo...,0
1572,"Yeah, like it or not Like a ball and a chain A...",0
2627,You ought to take us with you when you leave B...,3
819,"Well, the stairs sound so lonely without you A...",2


In [46]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['lyrics'], padding='max_length', truncation=True)

train_encodings = tokenizer(list(train_df['lyrics']), padding=True, truncation=True)
val_encodings = tokenizer(list(val_df['lyrics']), padding=True, truncation=True)


In [47]:
import torch

class LyricsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = LyricsDataset(train_encodings, train_df['sentiment'].tolist())
val_dataset = LyricsDataset(val_encodings, val_df['sentiment'].tolist())


In [48]:
from datasets import load_metric

# Load accuracy metric
accuracy_metric = load_metric("accuracy")

def compute_metrics(p):
    logits, labels = p
    predictions = np.argmax(logits, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)


/Users/adammillington/anaconda3/lib/python3.11/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [53]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


  0%|          | 0/234 [00:00<?, ?it/s]

{'loss': 1.6379, 'grad_norm': 4.786650657653809, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.13}
{'loss': 1.6134, 'grad_norm': 6.1113409996032715, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.26}
{'loss': 1.5984, 'grad_norm': 7.915622234344482, 'learning_rate': 3e-06, 'epoch': 0.38}
{'loss': 1.6087, 'grad_norm': 3.0257174968719482, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.51}
{'loss': 1.5566, 'grad_norm': 3.570732593536377, 'learning_rate': 5e-06, 'epoch': 0.64}
{'loss': 1.5599, 'grad_norm': 4.370570659637451, 'learning_rate': 6e-06, 'epoch': 0.77}
{'loss': 1.5769, 'grad_norm': 4.669127941131592, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.9}


  0%|          | 0/20 [00:00<?, ?it/s]

{'eval_loss': 1.5577938556671143, 'eval_accuracy': 0.2980769230769231, 'eval_runtime': 5.5432, 'eval_samples_per_second': 56.285, 'eval_steps_per_second': 3.608, 'epoch': 1.0}
{'loss': 1.5579, 'grad_norm': 3.9423305988311768, 'learning_rate': 8.000000000000001e-06, 'epoch': 1.03}
{'loss': 1.5701, 'grad_norm': 4.4329657554626465, 'learning_rate': 9e-06, 'epoch': 1.15}
{'loss': 1.5166, 'grad_norm': 7.610838890075684, 'learning_rate': 1e-05, 'epoch': 1.28}
{'loss': 1.5468, 'grad_norm': 3.419715642929077, 'learning_rate': 1.1000000000000001e-05, 'epoch': 1.41}
{'loss': 1.573, 'grad_norm': 3.788599967956543, 'learning_rate': 1.2e-05, 'epoch': 1.54}
{'loss': 1.5167, 'grad_norm': 5.615087509155273, 'learning_rate': 1.3000000000000001e-05, 'epoch': 1.67}
{'loss': 1.5646, 'grad_norm': 3.447225332260132, 'learning_rate': 1.4000000000000001e-05, 'epoch': 1.79}
{'loss': 1.5458, 'grad_norm': 4.769131183624268, 'learning_rate': 1.5e-05, 'epoch': 1.92}


  0%|          | 0/20 [00:00<?, ?it/s]

{'eval_loss': 1.544020414352417, 'eval_accuracy': 0.2916666666666667, 'eval_runtime': 5.3788, 'eval_samples_per_second': 58.005, 'eval_steps_per_second': 3.718, 'epoch': 2.0}
{'loss': 1.5104, 'grad_norm': 7.178765773773193, 'learning_rate': 1.6000000000000003e-05, 'epoch': 2.05}
{'loss': 1.4983, 'grad_norm': 3.8931076526641846, 'learning_rate': 1.7000000000000003e-05, 'epoch': 2.18}
{'loss': 1.4822, 'grad_norm': 4.603208065032959, 'learning_rate': 1.8e-05, 'epoch': 2.31}
{'loss': 1.5089, 'grad_norm': 4.2160444259643555, 'learning_rate': 1.9e-05, 'epoch': 2.44}
{'loss': 1.5926, 'grad_norm': 6.355203151702881, 'learning_rate': 2e-05, 'epoch': 2.56}
{'loss': 1.5333, 'grad_norm': 6.801198482513428, 'learning_rate': 2.1e-05, 'epoch': 2.69}
{'loss': 1.4435, 'grad_norm': 4.643118858337402, 'learning_rate': 2.2000000000000003e-05, 'epoch': 2.82}
{'loss': 1.5125, 'grad_norm': 4.888322353363037, 'learning_rate': 2.3000000000000003e-05, 'epoch': 2.95}


  0%|          | 0/20 [00:00<?, ?it/s]

{'eval_loss': 1.4874439239501953, 'eval_accuracy': 0.34294871794871795, 'eval_runtime': 5.7027, 'eval_samples_per_second': 54.71, 'eval_steps_per_second': 3.507, 'epoch': 3.0}
{'train_runtime': 250.1534, 'train_samples_per_second': 14.931, 'train_steps_per_second': 0.935, 'train_loss': 1.5470831129286025, 'epoch': 3.0}


TrainOutput(global_step=234, training_loss=1.5470831129286025, metrics={'train_runtime': 250.1534, 'train_samples_per_second': 14.931, 'train_steps_per_second': 0.935, 'total_flos': 982746262103040.0, 'train_loss': 1.5470831129286025, 'epoch': 3.0})

In [29]:
results = trainer.evaluate()
print(results)

  0%|          | 0/40 [00:00<?, ?it/s]

{'eval_loss': 4.488988399505615, 'eval_accuracy': 0.03645007923930269, 'eval_runtime': 12.591, 'eval_samples_per_second': 50.115, 'eval_steps_per_second': 3.177, 'epoch': 3.0}


In [32]:
from transformers import Trainer, TrainingArguments, BertForSequenceClassification
from sklearn.model_selection import ParameterGrid

# Define the parameter grid
param_grid = {
    'learning_rate': [2e-5, 3e-5, 5e-5],
    'per_device_train_batch_size': [8, 16, 32],
    'num_train_epochs': [3, 4, 5],
    'warmup_steps': [0, 500],
    'weight_decay': [0.01, 0.001]
}

# Create a list to store the results
results = []

# Iterate over all combinations of hyperparameters
for params in ParameterGrid(param_grid, ):
    training_args = TrainingArguments(
        output_dir='./results',
        learning_rate=params['learning_rate'],
        per_device_train_batch_size=params['per_device_train_batch_size'],
        per_device_eval_batch_size=params['per_device_train_batch_size'],
        num_train_epochs=params['num_train_epochs'],
        warmup_steps=params['warmup_steps'],
        weight_decay=params['weight_decay'],
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch"
    )

    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=99)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_results = trainer.evaluate()
    eval_results['params'] = params
    results.append(eval_results)

# Print the best hyperparameters
best_result = min(results, key=lambda x: x['eval_loss'])
print("Best hyperparameters:", best_result['params'])
print("Best evaluation loss:", best_result['eval_loss'])


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/948 [00:00<?, ?it/s]

{'loss': 4.5801, 'grad_norm': 8.692593574523926, 'learning_rate': 1.978902953586498e-05, 'epoch': 0.03}
{'loss': 4.6026, 'grad_norm': 6.65548849105835, 'learning_rate': 1.957805907172996e-05, 'epoch': 0.06}
{'loss': 4.5784, 'grad_norm': 8.007187843322754, 'learning_rate': 1.936708860759494e-05, 'epoch': 0.09}
{'loss': 4.6495, 'grad_norm': 8.252561569213867, 'learning_rate': 1.9156118143459917e-05, 'epoch': 0.13}
{'loss': 4.6157, 'grad_norm': 7.027004718780518, 'learning_rate': 1.8945147679324897e-05, 'epoch': 0.16}
{'loss': 4.6453, 'grad_norm': 6.201651573181152, 'learning_rate': 1.8734177215189874e-05, 'epoch': 0.19}
{'loss': 4.5781, 'grad_norm': 5.533116340637207, 'learning_rate': 1.8523206751054853e-05, 'epoch': 0.22}
{'loss': 4.6104, 'grad_norm': 5.384045124053955, 'learning_rate': 1.8312236286919833e-05, 'epoch': 0.25}
{'loss': 4.5855, 'grad_norm': 5.016408443450928, 'learning_rate': 1.8101265822784813e-05, 'epoch': 0.28}
{'loss': 4.5836, 'grad_norm': 5.545433044433594, 'learning_

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 4.551388740539551, 'eval_accuracy': 0.01901743264659271, 'eval_runtime': 11.4352, 'eval_samples_per_second': 55.181, 'eval_steps_per_second': 6.909, 'epoch': 1.0}
{'loss': 4.5396, 'grad_norm': 5.505777835845947, 'learning_rate': 1.3248945147679326e-05, 'epoch': 1.01}
{'loss': 4.5185, 'grad_norm': 5.721272945404053, 'learning_rate': 1.3037974683544306e-05, 'epoch': 1.04}
{'loss': 4.4947, 'grad_norm': 5.639856338500977, 'learning_rate': 1.2827004219409284e-05, 'epoch': 1.08}
{'loss': 4.4905, 'grad_norm': 5.564944267272949, 'learning_rate': 1.2616033755274262e-05, 'epoch': 1.11}
{'loss': 4.5117, 'grad_norm': 6.717665672302246, 'learning_rate': 1.240506329113924e-05, 'epoch': 1.14}
{'loss': 4.5348, 'grad_norm': 5.911284923553467, 'learning_rate': 1.219409282700422e-05, 'epoch': 1.17}
{'loss': 4.5365, 'grad_norm': 6.411997318267822, 'learning_rate': 1.19831223628692e-05, 'epoch': 1.2}
{'loss': 4.5199, 'grad_norm': 6.429394721984863, 'learning_rate': 1.1772151898734178e-05, 'ep

...

  0%|          | 0/79 [00:00<?, ?it/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/948 [00:00<?, ?it/s]

{'loss': 4.6903, 'grad_norm': 8.00285816192627, 'learning_rate': 1.978902953586498e-05, 'epoch': 0.03}
{'loss': 4.6399, 'grad_norm': 6.19515323638916, 'learning_rate': 1.957805907172996e-05, 'epoch': 0.06}
{'loss': 4.5861, 'grad_norm': 6.1281962394714355, 'learning_rate': 1.936708860759494e-05, 'epoch': 0.09}


KeyboardInterrupt: 

In [33]:
results


[{'eval_loss': 4.509838581085205,
  'eval_accuracy': 0.02377179080824089,
  'eval_runtime': 15.2409,
  'eval_samples_per_second': 41.402,
  'eval_steps_per_second': 5.183,
  'epoch': 3.0,
  'params': {'learning_rate': 2e-05,
   'num_train_epochs': 3,
   'per_device_train_batch_size': 8,
   'warmup_steps': 0,
   'weight_decay': 0.01}}]